In [1]:
import csv, os
import matplotlib.pyplot as plt
import numpy as np

def read_csv(dir, run, d=1):
    filename = os.path.join(dir, run, "logs.csv")
    steps, eps_ret, eps_n = [], [], []

    with open(filename, 'r') as file:
        csvreader = csv.reader(file)
        header = next(csvreader)
        for row in csvreader:            
            if len(row) > 0 and int(row[-1]) > 100:
                steps.append(int(row[2])/d)
                eps_ret.append(float(row[-2]))
                #eps_n.append(row[-1])
    return steps, eps_ret

def simplify(steps, eps_ret, gap=100000):
    ind = 0
    new_steps, new_eps_ret = [], []
    for n, (i, j) in enumerate(zip(steps, eps_ret)):
        if i > ind:
            new_steps.append(i)
            new_eps_ret.append(j)
            ind += gap
    return new_steps, new_eps_ret
    
def mv(a, n=1000) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n    

In [ ]:
servers = ["uk", "cam", "v1", "v2", "v3", "v4"]
latest_runs = ["lstm_3_1_rec_n_5_gbste", 
               "lstm_3_1_rec_n_5_rei",
               "lstm_3_1_rec_n_5_ste",
               "lstm_attn_3_1_rec_n_10_rei",
               "lstm_3_1_rec_n_5_gbste_2",
               "lstm_3_1_rec_n_5_rei_2",]               

for server, lr in zip(servers, latest_runs):
    !mkdir ~/RS/thinker/logs/planner_logs/$lr
    !scp $server:~/RS/thinker/logs/torchbeast/latest/*.csv ~/RS/thinker/logs/planner_logs/$lr

In [2]:
#%matplotlib inline
%matplotlib qt

runs = os.listdir('logs/planner_logs')
steps, eps_rets = {}, {}
mv_avg = 20

for run in runs:
    if "." in run or "past" == run: continue
    if "rei" in run:
        st = run.find("rec_n_")
        d = int(run[st+6:run.find("_", st+6)])
    else:
        d = 1  
    step, eps_ret = simplify(*read_csv('logs/planner_logs', run, d))
    steps[run] = step
    eps_rets[run] = eps_ret

f, ax = plt.subplots(1, 1, figsize=(12,7))

linestyles=["solid", "dashed", "dotted","dashdot"]
for n, run in enumerate(eps_rets.keys()):         
    ax.plot(np.array(steps[run][mv_avg-1:]), 
            mv(eps_rets[run], n=mv_avg), 
            label=run, 
            linestyle=linestyles[n//12])
plt.xlabel('Step')
plt.ylabel('Eps return')
plt.axhline(y=0.12, color='r', label="base policy")
plt.axhline(y=1.08, color='g', label="one-step greedy search")
plt.axhline(y=1.17, color='y', label="two-step greedy search")

plt.legend()
plt.show()    

libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast
